# size

> Size modules for benchmarking

In [ ]:
#| default_exp size

In [ ]:
#| export
from fasterbench.core import _bytes_to_mib
import torch
import io
from dataclasses import dataclass, asdict
from typing import Any, Dict, Iterable, List, Mapping, Sequence, Tuple, Union

In [ ]:
#| export
def get_model_size(model: torch.nn.Module) -> int:
    """Return the *on‑disk* size (in **bytes**) of the serialised *model*."""
    buf = io.BytesIO()
    try:
        model.save(buf)
    except Exception:
        torch.save(model.state_dict(), buf)
    return buf.getbuffer().nbytes

#| export
def get_num_parameters(model: torch.nn.Module, trainable_only: bool = True) -> int:
    """Count the number of (optionally trainable) parameters."""
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())

#| export
@dataclass(slots=True)
class SizeMetrics:
    disk_bytes: int
    size_mib: float
    num_params: int

    def as_dict(self) -> Dict[str, float]:
        return asdict(self)

#| export
def compute_size(model: torch.nn.Module) -> SizeMetrics:
    """High‑level helper that returns a **SizeMetrics** instance."""
    disk = get_model_size(model)
    params = get_num_parameters(model)
    return SizeMetrics(disk_bytes=disk, size_mib=_bytes_to_mib(disk), num_params=params)